In [2]:
!pip install selenium

# 필요한 패키지 불러오기

In [3]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pickle

# 호텔별 URL을 가져오기

In [4]:
# url 함수
def make_url(Url):
    import time
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome()
    driver.get(Url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    click1 = driver.find_element(By.XPATH,'//*[@id="__next"]/div/header/section/div/div[1]/section[2]/div/div/div/div/div[2]/div/div[1]/span[1]/span')
    click1.click()
    time.sleep(6)
    click2 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[1]')
    click2.click()
    click3 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[2]')
    click3.click()
    time.sleep(2)
    click4 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/footer/div/button[2]')
    click4.click()
    time.sleep(2)
    #무한 스크롤
    
    time.sleep(1)
    
    SCROLL_PAUSE_TIME = 1.5
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height  

    soup = BeautifulSoup(html, 'html.parser')
    data = soup.select("section > div > div > a")
        
    for d in data:
        hotelurl.append(d.attrs['href'])
    # driver.close()
  

In [8]:
hotelurl = []
make_url("https://www.yanolja.com/hotel/r-900590?pageKey=1724956175263")

# 리뷰 수집

In [12]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

hotel = []
star = []
reviews = []
rev_sum_pos = []
rev_sum_neg = []
columns = ['hotel','star' ,'reviews', 'Review Summary Pos', 'Review Summary Neg','url' ]   


for url in hotelurl:
    cnt = 0
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(1)
    Url = "https://www.yanolja.com"+url
    driver.get(Url)
    WebDriverWait(driver, 10)
    # try:
    # # 요소 찾기 시도
    try : 
        rev_sum_pos.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text)
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except NoSuchElementException:
        rev_sum_pos.append("")
    try:   
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except TimeoutException:
        pass
    try :    
        rev_sum_neg.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text) 
    except NoSuchElementException:
        rev_sum_neg.append("") 
    # except NoSuchElementException:
    # # 요소가 없을 경우 공백 문자열 설정
    #     rev_sum_pos = ''
    #     rev_sum_neg = ''
    
    hotel.append(driver.find_element(By.CSS_SELECTOR,'#__next > div > div > main > article > div.css-1cc3d9 > div > div.css-1e56347 > div.css-11vo59c > h1').text)
    
    # 리뷰 페이지 클릭 (Click review page)
    try:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-xa55ms'))
        )
    except NoSuchElementException :
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    except TimeoutException:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    element2.click()
    # 무한 스크롤
    import time
    time.sleep(1)
    
    # SCROLL_PAUSE_TIME = 1.5
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "css-stl2cn")))
    element.click()
    
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[text()='평점 낮은순']")))
    element.click()
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    max = int(driver.find_element(By.CLASS_NAME,"css-12rhouj").text.replace("(","").replace(")",""))
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if max > 1000:
            if(cnt>20):
                break
            cnt +=1
        if new_height == last_height:
            break
    
        last_height = new_height
    
    # 스크롤 끝나고 페이지 elements 가져오기
    star.append(float(driver.find_element(By.CLASS_NAME, 'css-u41ekg').text))
    soup = bs(driver.page_source, "lxml")
    
    # 특정 클래스 이름을 가진 모든 요소 찾기
    elements = soup.find_all('p')
    temp = ""
    # 데이터 추출 
    review = driver.find_elements(By.CLASS_NAME,"css-c92dc4")
    for i in review:
        temp += i.text
    reviews.append(temp)     
    
df = pd.DataFrame({"Hotel" : hotel, "Star" : star ,"Reviews" : reviews, "Review Pos" : rev_sum_pos,"Review Neg" : rev_sum_neg,"Url": hotelurl})   
df["Url"] ="https://www.yanolja.com"+df["Url"]

In [14]:
with open('cc.pickle', 'wb') as f:
    pickle.dump(df, f)

In [16]:
df

,Hotel,Star,Reviews,Review Pos,Review Neg,Url
0,비체팰리스,4.3,14시 입실했으나 청소가 안되어 2시간 이상 기다렸음. 그에 대해 죄송하다는 말로 ...,"바다 바로 앞에 위치한 숙소로, 가족 여행에 적합한 환경을 제공합니다. 스위트룸은 ...","숙소는 바다에 인접해 있어 멋진 전망을 제공하지만, 전반적으로 노후된 시설과 청결 ...",https://www.yanolja.com/hotel/1000112956
1,호텔어라이브 태안 탼 한옥비치리조트,4.6,숙소 자체는 아주 예쁘고 신설이라 좋아요\n다만\n벌레가 많고 \n비오면 불멍하기 ...,"깨끗한 한옥팬션으로, 바다에서 5분 거리에 위치해 있습니다. 넓고 깔끔한 숙소 내부...",,https://www.yanolja.com/hotel/10054394
2,신라스테이 천안,4.6,최악..\n새벽4시 창가벽쪽 콘센트에서 크게 딸깍하는소리가 1시간가량나서 4시부터 ...,"전반적으로 깔끔하고 쾌적한 환경을 제공하며, 조식이 맛있고 직원의 친절함이 강조되고...",다양한 리뷰를 통해 전반적인 만족도와 불만족도가 혼재되어 있는 것으로 보입니다. 주...,https://www.yanolja.com/hotel/3001693
3,청풍리조트,4.3,뷰는 너무좋은곳이네요\n객실 노후는 어쩔수없지만\n침대위에 머리카락이며 구석바닥에 ...,"호수 전망이 뛰어난 숙소로, 청결 상태와 가성비가 좋다는 후기가 많습니다. 최근 리...","멋진 호수뷰와 편안한 침구로 알려져 있으며, 실내 수영장과 케이블카, 모노레일과의 ...",https://www.yanolja.com/hotel/3013094
4,대천 케이 호텔,4.2,생각보다는 별로였음너무지저분합니다\n냄새도나고 위에 소리도시끄러워 한숨도 못자고 왔...,"숙소는 대천해수욕장 근처에 위치하고 있으며, 바다와 가까운 점이 장점으로 언급됩니다...",위치는 우수하나 주차공간이 부족하다는 의견이 많습니다. 주차 문제로 인해 인근 공용...,https://www.yanolja.com/hotel/3001638
5,제이원 호텔,4.7,입실하여 씻고 누웠는데 초인종이 울리고 문이열림(직원이라함-점검중)\n이건 무슨 경...,"청결한 상태를 유지하고 있으며, 친절한 직원 서비스가 특징입니다. 객실은 조용하고 ...","청주 인근에 위치한 비즈니스 호텔이자 레지던스형으로, 간단한 취사와 세탁이 가능한 ...",https://www.yanolja.com/hotel/1000110447
6,호텔 캄캄,4.7,깨끗한 곳을 찾고 찾고 찾아서 갔는데..\n이거 베드버그일까요?\n소독 하셔야 할 ...,"청결과 편안함이 두드러진 장소로, 많은 이용객이 만족감을 표현하고 있습니다. 직원의...","전체적으로 깔끔한 객실과 잘 관리된 욕실을 제공하며, 방음과 침구 품질이 우수하다는...",https://www.yanolja.com/hotel/1000109243
7,대천 환상의 바다 리조트,4.1,D동숙소 최악입니다\n무슨 리조트가 을씨년스럽고\n문열자 오래된시골집냄새\n90년대...,해변 근처에 위치한 숙소는 뛰어난 바다 전망과 청결한 내부로 만족도가 높습니다. 객...,"멋진 바다 뷰와 가격 대비 장점이 있으나, 여러 리뷰에서 청결 문제가 자주 언급되고...",https://www.yanolja.com/hotel/3018453
8,베니키아 테크노밸리호텔,4.6,화장실문도 안닫히고...\n있는물건도 없고...\n가격대비 너무 심하네요어차피 갈일...,"전반적으로 청결하고 쾌적한 환경을 제공하며, 직원들의 친절한 서비스를 자랑합니다. ...",직원의 친절함과 주변의 편의시설이 잘 갖추어져 있어 접근성이 좋습니다. 청결도와 시...,https://www.yanolja.com/hotel/3000753
9,천안 뷰 호텔(Hotel),4.5,불친절하고\n불편해요청소 상태 불양 에어컨 안나와서 틀어 달라고 하니꺈 짜증내면서 ...,"본 숙소는 가격 대비 우수한 가성비를 제공하며, 청결하고 조용한 환경이 특징입니다....","가격 대비 가성비가 좋고 방이 깔끔하다는 긍정적인 평가가 있지만, 청결 상태에 대한...",https://www.yanolja.com/hotel/3020578
